### Configuration
- Vector Store : ChromaDB
- QA model : Ollama.LLAMA3.2
- Embedding model : Nomic Embed text
- Split method : Langchain.TokenTextSplitter CS : 4900 Overlap 200 previously 10000

### Imports

In [2]:
from langchain_chroma import Chroma
from langchain_ollama import OllamaEmbeddings
from langchain_ollama.llms import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate

from tqdm import tqdm
from utils.my_indexing_utils import *
from langchain.text_splitter import TokenTextSplitter
from langchain_core.prompts import ChatPromptTemplate


#### Set parameters


In [3]:
CHUNK_SIZE = 4900
OVERLAP = 200
EMBEDDING_MODEL="nomic-embed-text"
DOCUMENTS_DIR="../Data/corpus"
SPLITTED_DOCS_DIR="splitted_docs"
PERSISTENT_DIR="chroma_langchain_db"
SAMPLE_SIZE=1
VERBOSE=True
BATCH_SIZE = 4

## Indexing

### Set embedding model

In [4]:
embeddings = OllamaEmbeddings(
    model=EMBEDDING_MODEL,
)

### Split texts and save them

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=OVERLAP)
documents_list = makeDocuments(path=DOCUMENTS_DIR,sample_size=SAMPLE_SIZE,verbose=VERBOSE)
all_splits = split_docs(text_splitter=text_splitter,documents_list=documents_list,verbose=VERBOSE)
save_documents(all_splits,output_dir=SPLITTED_DOCS_DIR,verbose=VERBOSE)

### Set if not exist a vector store

In [6]:
persist_dir = PERSISTENT_DIR
vector_store = make_db(persist_dir=PERSISTENT_DIR,verbose=VERBOSE,embeddings=embeddings)

Vector store déjà présent, chargement...


### Embbed doc & store them in database once

In [24]:
def indexing():
    indexed_ids = vector_store.get(include=[])["ids"]
    if len(indexed_ids) == len(all_splits):
        print("Already indexed...")
    else :
        batch_size = BATCH_SIZE
        for i in tqdm(range(0, len(all_splits), batch_size), desc="Vectorizing",colour="cyan"):
            batch = all_splits[i:i+batch_size]
            vector_store.add_documents(batch)
indexing()

Vectorizing: 100%|██████████| 225/225 [03:45<00:00,  1.00s/it]


## Retrieval

In [21]:
def retrieve_context(query, k=3, separator="\n\n"):
    docs = vector_store.similarity_search_with_score(query=query, k=k)
    scores = []
    context = ""
    for doc, score in docs:
        scores.append(score)
        context += f"{doc.page_content}{separator}"  
    return context.strip(), scores, docs

In [ ]:
query = """Que signifie une “interpellation” dans le contexte parlementaire du XIXe siècle ?"""
context, scores, docs = retrieve_context(query=query)

prompt = ChatPromptTemplate.from_messages([
    ("system", "Réponds à la question en utilisant le contexte suivant : {context}"),
    ("user", "Utilise ce contexte pour répondre à la question suivante : {question}")
])

model = OllamaLLM(model="llama3.2")

chain = prompt | model

response = chain.invoke({
    "question": query,
    "context": context
})
print(response)

Dans le contexte parlementaire du XIXe siècle, une "interpellation" désigne un amendement ou une motion proposée par un député pour examiner ou discuter d'un sujet particulier, souvent lié à une question politique, sociale ou économique.

Dans ce sens, l'interpellation est un moyen permettant aux députés de faire appel au vote du Parlement sur une question spécifique. Lorsqu'une interpellation est proposée, elle doit passer par plusieurs étapes, notamment une motion d'examen (ou "de lecture") qui permet de discuter de la question, puis un vote sur l'amendement ou le projet de loi en question.

L'interpellation peut être présentée par n'importe quel député et portée par son collègue pour s'y opposer. C'est ainsi que se manifeste souvent la opposition politique au gouvernement ou à une décision prise par les responsables politiques.

Dans le contexte du texte fourni, on voit que plusieurs députés ont présenté des interpellations sur divers sujets, notamment liés aux indemnités accordées 

In [23]:
print(docs)

[(Document(id='d8690812-7d92-4f65-a883-70d66aa7909c', metadata={'chunk_id': '1882-06-19_1_chunk_15', 'parent_source': '1882-06-19_1', 'source': '1882-06-19_1'}, page_content="enciennes, Nord). Leroy (Arthur). Logerotte.\n\nRibot. Sarrien. Turquet Wilson.\n\nABSENTS PAR, CONGÉ : MM. Andrieux. Beauquier. Bertholon. Bourrillon. Chantemille. Daynaud. Devès. Douville-Maillefeu (comte de). Duchesne-Fournet.\n\nDucroz. Fauré. Fourcand. Galpin. La Bassetière (de). Laffitte de Lajoannenque (de).\n\nLaurençon. Leliôvre (Adolphe). Le Roux.\n\nLoubet. Marmottan. Mézières. Mir. Pellet (Marcellin). Récipon. Roys (marquis de). Sourigues. Vernhes.\n\nM. Bovier-Lapierre, porté comme s'étant abstenu dans le scrutin du samedi 17 juin sur l'in-\n\nterdiction du compte rendu des débats des procès en divorce déclare avoir voté « contre » l'in.\n\nterdiction.\n\nM Drumel déclare que, dans la séance du samedi 17 juin, il a voté « pour» l'amendement de M. Gatineau sur la publicité des débats des procès en divo